In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("MiApp").getOrCreate()

24/11/25 21:19:50 WARN Utils: Your hostname, Isa resolves to a loopback address: 127.0.1.1; using 172.21.238.159 instead (on interface eth0)
24/11/25 21:19:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/25 21:19:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark = SparkSession.builder \
    .appName("Cargar Archivos CSV") \
    .config("spark.sql.warehouse.dir", "file:///C:/tmp/spark-warehouse") \
    .config("spark.security.manager.enabled", "false") \
    .config("spark.hadoop.security.authentication", "none") \
    .getOrCreate()

24/11/25 21:20:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [1]:
import pandas as pd

In [5]:
# Ruta a los archivos
sales_path = "../data/raw/sales.csv"
product_hierarchy_path = "../data/raw/product_hierarchy.csv"
store_cities_path = "../data/raw/store_cities.csv"

# Cargar los archivos CSV
df_sales = spark.read.csv(sales_path, header=True, inferSchema=True)
df_product_hierarchy = spark.read.csv(product_hierarchy_path, header=True, inferSchema=True)
df_store_cities = spark.read.csv(store_cities_path, header=True, inferSchema=True)

In [6]:
df_sales.show()

+----------+--------+----------+-----+-------+-----+-----+------------+-----------+------------+-----------+----------------+---------------------+
|product_id|store_id|      date|sales|revenue|stock|price|promo_type_1|promo_bin_1|promo_type_2|promo_bin_2|promo_discount_2|promo_discount_type_2|
+----------+--------+----------+-----+-------+-----+-----+------------+-----------+------------+-----------+----------------+---------------------+
|     P0001|   S0002|2017-01-02|  0.0|    0.0|  8.0| 6.25|        PR14|       NULL|        PR03|       NULL|            NULL|                 NULL|
|     P0001|   S0012|2017-01-02|  1.0|    5.3|  0.0| 6.25|        PR14|       NULL|        PR03|       NULL|            NULL|                 NULL|
|     P0001|   S0013|2017-01-02|  2.0|  10.59|  0.0| 6.25|        PR14|       NULL|        PR03|       NULL|            NULL|                 NULL|
|     P0001|   S0023|2017-01-02|  0.0|    0.0|  6.0| 6.25|        PR14|       NULL|        PR03|       NULL|    

In [70]:
df_product_hierarchy.head()

,product_id,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id
0,P0000,5.0,20.0,12.0,NaN,H00,H0004,H000401,H00040105,H0004010534
1,P0001,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
2,P0002,22.0,40.0,22.0,cluster_0,H03,H0315,H031508,H03150800,H0315080028
3,P0004,2.0,13.0,4.0,cluster_3,H03,H0314,H031405,H03140500,H0314050003
4,P0005,16.0,30.0,16.0,cluster_9,H03,H0312,H031211,H03121109,H0312110917


In [71]:
df_store_cities.head()

,store_id,storetype_id,store_size,city_id
0,S0091,ST04,19,C013
1,S0012,ST04,28,C005
2,S0045,ST04,17,C008
3,S0032,ST03,14,C019
4,S0027,ST04,24,C022


## **Exploración de datos**

In [65]:
total_decimal_sales = len(df_sales[df_sales['sales'] % 1 != 0])
print(f"Total de registros con decimales en 'sales': {total_decimal_sales}")

Total de registros con decimales en 'sales': 1218907


In [67]:
total_records = len(df_sales)
print(f"Total de registros en el dataset: {total_records}")

Total de registros en el dataset: 19454838


In [64]:
# Filtrar las filas con valores decimales en 'sales'
decimal_sales = df_sales[df_sales['sales'] % 1 != 0]

# Obtener los product_id únicos con este problema
products_with_decimal_sales = decimal_sales['product_id'].unique()

# Convertir a un DataFrame para un análisis más fácil
products_with_decimal_sales_df = pd.DataFrame(products_with_decimal_sales, columns=['product_id'])

# Agregar conteo de registros por product_id
product_decimal_count = decimal_sales['product_id'].value_counts().reset_index()
product_decimal_count.columns = ['product_id', 'count']

# Mostrar los primeros valores
print(product_decimal_count.head(100))

# Número total de productos con ventas decimales
print(f"Número total de productos con ventas decimales: {len(products_with_decimal_sales)}")


   product_id  count
0       P0413  13278
1       P0663   8185
2       P0125   8173
3       P0131   8171
4       P0333   8156
..        ...    ...
95      P0477   4589
96      P0054   4583
97      P0039   4554
98      P0087   4545
99      P0217   4517

[100 rows x 2 columns]
Número total de productos con ventas decimales: 605


In [9]:
df_store_cities.head()

,store_id,storetype_id,store_size,city_id
0,S0091,ST04,19,C013
1,S0012,ST04,28,C005
2,S0045,ST04,17,C008
3,S0032,ST03,14,C019
4,S0027,ST04,24,C022


In [55]:
# Contar las frecuencias de los valores únicos en la columna 10
value_counts = (
    df_sales.iloc[:, 3]
    .value_counts(dropna=False)  # Incluye NaN si los hay
    .reset_index()  # Convierte a DataFrame
)
value_counts.columns = ['Unique Value', 'Count']  # Renombrar columnas

# Ordenar por frecuencia de mayor a menor
value_counts_sorted = value_counts.sort_values(by='Count', ascending=False)

value_counts_sorted.head(100)

,Unique Value,Count
0,0.000,15074394
1,1.000,1794179
2,NaN,1190762
3,2.000,621461
4,3.000,266242
...,...,...
94,68.000,51
96,0.308,50
97,0.232,50
98,0.256,49


In [57]:
import numpy as np

# Filtrar filas donde sales tiene un número decimal
df_sales_with_decimals = df_sales[np.modf(df_sales['sales'])[0] != 0]

df_sales_with_decimals.head()


,product_id,store_id,date,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_bin_2,promo_discount_2,promo_discount_type_2
5335,P0298,S0123,2017-01-02,0.530,3.68,6.640,7.49,PR14,NaN,PR03,NaN,NaN,NaN
5336,P0298,S0139,2017-01-02,1.414,9.81,28.428,7.49,PR14,NaN,PR03,NaN,NaN,NaN
5469,P0316,S0062,2017-01-02,2.290,127.00,0.000,59.90,PR14,NaN,PR03,NaN,NaN,NaN
5470,P0316,S0085,2017-01-02,2.580,143.09,0.000,59.90,PR14,NaN,PR03,NaN,NaN,NaN
7270,P0413,S0001,2017-01-02,1.695,3.12,4.415,1.99,PR03,verylow,PR03,NaN,NaN,NaN


# **Análisis para limpieza**

In [59]:
decimal_sales = df_sales[df_sales['sales'] % 1 != 0]
decimal_sales.head()

,product_id,store_id,date,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_bin_2,promo_discount_2,promo_discount_type_2
5335,P0298,S0123,2017-01-02,0.530,3.68,6.640,7.49,PR14,NaN,PR03,NaN,NaN,NaN
5336,P0298,S0139,2017-01-02,1.414,9.81,28.428,7.49,PR14,NaN,PR03,NaN,NaN,NaN
5469,P0316,S0062,2017-01-02,2.290,127.00,0.000,59.90,PR14,NaN,PR03,NaN,NaN,NaN
5470,P0316,S0085,2017-01-02,2.580,143.09,0.000,59.90,PR14,NaN,PR03,NaN,NaN,NaN
7270,P0413,S0001,2017-01-02,1.695,3.12,4.415,1.99,PR03,verylow,PR03,NaN,NaN,NaN


In [60]:
print(f"Porcentaje de valores decimales: {len(decimal_sales) / len(df_sales) * 100:.2f}%")

Porcentaje de valores decimales: 6.27%


In [11]:
df_sales['date'] = pd.to_datetime(df_sales['date'])

In [61]:
# Filtrar los valores decimales
decimal_sales = df_sales[df_sales['sales'] % 1 != 0]

# Crear una nueva columna para el rango de cada valor decimal
decimal_sales['range'] = pd.cut(
    decimal_sales['sales'],
    bins=[-float('inf'), 0, 1, 2, 3, 4, 5, 6, float('inf')],
    labels=["Menor a 0", "Entre 0 y 1", "Entre 1 y 2", "Entre 2 y 3", "Entre 3 y 4", "Entre 4 y 5", "Entre 5 y 6", "Mayor a 6"]
)

# Contar la cantidad de valores en cada rango
decimal_analysis = decimal_sales['range'].value_counts().reset_index()
decimal_analysis.columns = ['Range', 'Count']

# Mostrar el resultado
print(decimal_analysis)


         Range  Count
0  Entre 0 y 1  12201
1  Entre 1 y 2   5674
2    Mayor a 6   4152
3  Entre 3 y 4   2108
4  Entre 2 y 3   2053
5  Entre 4 y 5   1050
6  Entre 5 y 6    907
7    Menor a 0      0


C:\Users\zantr\AppData\Local\Temp\ipykernel_21048\788316473.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  decimal_sales['range'] = pd.cut(


## **Productos más populares cada mes durante el año 2018**

In [32]:
df_2018 = df_sales[df_sales['date'].dt.year == 2018].copy()
df_2018['month'] = df_2018['date'].dt.month_name()
df_2018['month_num'] = df_2018['date'].dt.month

In [34]:
product_sales = (
    df_2018.groupby(['month_num','month', 'product_id'])['sales']
    .sum()
    .reset_index()
    .sort_values(['month_num', 'sales'], ascending=[True, False])
)

In [37]:
top_5_products_by_month = (
    product_sales.groupby('month')
    .head(5)
    .reset_index(drop=True)[['month', 'product_id', 'sales']]
)

In [68]:
top_5_products_by_month.head(10)

,month,product_id,sales
0,January,P0438,21326.0
1,January,P0103,19046.0
2,January,P0364,7984.0
3,January,P0051,7754.0
4,January,P0590,5692.0
5,February,P0438,21485.0
6,February,P0103,16474.0
7,February,P0051,7555.0
8,February,P0388,6212.0
9,February,P0590,5952.0


## **En cuántas tiendas se implementaron promociones del tipo 1 en el canal 1 durante el año 2019**

In [83]:
unique_store_ids_2019 = df_sales[(df_sales['promo_type_1'].notna()) 
& (df_sales['date'].dt.year == 2019)]['store_id'].nunique()

In [84]:
unique_store_ids_2019

144

## **Cuál fue la ciudad que tuvo el mayor volumen de ventas en general durante todo el periodo de 2017 a 2019**

In [90]:
columns_sales = ['store_id', 'date', 'sales', 'price', 'revenue']
columns_store_cities = ['store_id', 'city_id']

df_merged_sales_store_cities = pd.merge(
    df_sales[columns_sales],
    df_store_cities[columns_store_cities],
    on='store_id',
    how='left'
)

In [91]:
df_merged_sales_store_cities.head()

,store_id,date,sales,price,revenue,city_id
0,S0002,2017-01-02,0.0,6.25,0.00,C007
1,S0012,2017-01-02,1.0,6.25,5.30,C005
2,S0013,2017-01-02,2.0,6.25,10.59,C026
3,S0023,2017-01-02,0.0,6.25,0.00,C008
4,S0025,2017-01-02,0.0,6.25,0.00,C024


In [100]:
top_city = df_merged_sales_store_cities.groupby('city_id')['sales'].sum().idxmax()
print(f"Ciudad con las mayores ventas 2017 a 2019: {top_city}")

Ciudad con las mayores ventas 2017 a 2019: C014
